### Load test data into postgres for processing

In [1]:
# Import dependencied
import pandas as pd
import requests
from sqlalchemy import create_engine

In [2]:
test_data = pd.read_csv("../resources/fraudtest.csv")
test_data['cc_num'].count()

555719

In [3]:
test_data.head()
del test_data["Unnamed: 0"]
test_data

,trans_date_trans_time,cc_num,merchant,category,amt,first,last,gender,street,city,...,lat,long,city_pop,job,dob,trans_num,unix_time,merch_lat,merch_long,is_fraud
0,2020-06-21 12:14:25,2291163933867244,fraud_Kirlin and Sons,personal_care,2.86,Jeff,Elliott,M,351 Darlene Green,Columbia,...,33.9659,-80.9355,333497,Mechanical engineer,1968-03-19,2da90c7d74bd46a0caf3777415b3ebd3,1371816865,33.986391,-81.200714,0
1,2020-06-21 12:14:33,3573030041201292,fraud_Sporer-Keebler,personal_care,29.84,Joanne,Williams,F,3638 Marsh Union,Altonah,...,40.3207,-110.4360,302,"Sales professional, IT",1990-01-17,324cc204407e99f51b0d6ca0055005e7,1371816873,39.450498,-109.960431,0
2,2020-06-21 12:14:53,3598215285024754,"fraud_Swaniawski, Nitzsche and Welch",health_fitness,41.28,Ashley,Lopez,F,9333 Valentine Point,Bellmore,...,40.6729,-73.5365,34496,"Librarian, public",1970-10-21,c81755dbbbea9d5c77f094348a7579be,1371816893,40.495810,-74.196111,0
3,2020-06-21 12:15:15,3591919803438423,fraud_Haley Group,misc_pos,60.05,Brian,Williams,M,32941 Krystal Mill Apt. 552,Titusville,...,28.5697,-80.8191,54767,Set designer,1987-07-25,2159175b9efe66dc301f149d3d5abf8c,1371816915,28.812398,-80.883061,0
4,2020-06-21 12:15:17,3526826139003047,fraud_Johnston-Casper,travel,3.19,Nathan,Massey,M,5783 Evan Roads Apt. 465,Falmouth,...,44.2529,-85.0170,1126,Furniture designer,1955-07-06,57ff021bd3f328f8738bb535c302a31b,1371816917,44.959148,-85.884734,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
555714,2020-12-31 23:59:07,30560609640617,fraud_Reilly and Sons,health_fitness,43.77,Michael,Olson,M,558 Michael Estates,Luray,...,40.4931,-91.8912,519,Town planner,1966-02-13,9b1f753c79894c9f4b71f04581835ada,1388534347,39.946837,-91.333331,0
555715,2020-12-31 23:59:09,3556613125071656,fraud_Hoppe-Parisian,kids_pets,111.84,Jose,Vasquez,M,572 Davis Mountains,Lake Jackson,...,29.0393,-95.4401,28739,Futures trader,1999-12-27,2090647dac2c89a1d86c514c427f5b91,1388534349,29.661049,-96.186633,0
555716,2020-12-31 23:59:15,6011724471098086,fraud_Rau-Robel,kids_pets,86.88,Ann,Lawson,F,144 Evans Islands Apt. 683,Burbank,...,46.1966,-118.9017,3684,Musician,1981-11-29,6c5b7c8add471975aa0fec023b2e8408,1388534355,46.658340,-119.715054,0
555717,2020-12-31 23:59:24,4079773899158,fraud_Breitenberg LLC,travel,7.99,Eric,Preston,M,7020 Doyle Stream Apt. 951,Mesa,...,44.6255,-116.4493,129,Cartographer,1965-12-15,14392d723bb7737606b2700ac791b7aa,1388534364,44.470525,-117.080888,0


In [4]:
test_data['is_fraud'].value_counts()

0    553574
1      2145
Name: is_fraud, dtype: int64

In [5]:
genuine_df = test_data.loc[test_data['is_fraud']==0]
genuine_df

,trans_date_trans_time,cc_num,merchant,category,amt,first,last,gender,street,city,...,lat,long,city_pop,job,dob,trans_num,unix_time,merch_lat,merch_long,is_fraud
0,2020-06-21 12:14:25,2291163933867244,fraud_Kirlin and Sons,personal_care,2.86,Jeff,Elliott,M,351 Darlene Green,Columbia,...,33.9659,-80.9355,333497,Mechanical engineer,1968-03-19,2da90c7d74bd46a0caf3777415b3ebd3,1371816865,33.986391,-81.200714,0
1,2020-06-21 12:14:33,3573030041201292,fraud_Sporer-Keebler,personal_care,29.84,Joanne,Williams,F,3638 Marsh Union,Altonah,...,40.3207,-110.4360,302,"Sales professional, IT",1990-01-17,324cc204407e99f51b0d6ca0055005e7,1371816873,39.450498,-109.960431,0
2,2020-06-21 12:14:53,3598215285024754,"fraud_Swaniawski, Nitzsche and Welch",health_fitness,41.28,Ashley,Lopez,F,9333 Valentine Point,Bellmore,...,40.6729,-73.5365,34496,"Librarian, public",1970-10-21,c81755dbbbea9d5c77f094348a7579be,1371816893,40.495810,-74.196111,0
3,2020-06-21 12:15:15,3591919803438423,fraud_Haley Group,misc_pos,60.05,Brian,Williams,M,32941 Krystal Mill Apt. 552,Titusville,...,28.5697,-80.8191,54767,Set designer,1987-07-25,2159175b9efe66dc301f149d3d5abf8c,1371816915,28.812398,-80.883061,0
4,2020-06-21 12:15:17,3526826139003047,fraud_Johnston-Casper,travel,3.19,Nathan,Massey,M,5783 Evan Roads Apt. 465,Falmouth,...,44.2529,-85.0170,1126,Furniture designer,1955-07-06,57ff021bd3f328f8738bb535c302a31b,1371816917,44.959148,-85.884734,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
555714,2020-12-31 23:59:07,30560609640617,fraud_Reilly and Sons,health_fitness,43.77,Michael,Olson,M,558 Michael Estates,Luray,...,40.4931,-91.8912,519,Town planner,1966-02-13,9b1f753c79894c9f4b71f04581835ada,1388534347,39.946837,-91.333331,0
555715,2020-12-31 23:59:09,3556613125071656,fraud_Hoppe-Parisian,kids_pets,111.84,Jose,Vasquez,M,572 Davis Mountains,Lake Jackson,...,29.0393,-95.4401,28739,Futures trader,1999-12-27,2090647dac2c89a1d86c514c427f5b91,1388534349,29.661049,-96.186633,0
555716,2020-12-31 23:59:15,6011724471098086,fraud_Rau-Robel,kids_pets,86.88,Ann,Lawson,F,144 Evans Islands Apt. 683,Burbank,...,46.1966,-118.9017,3684,Musician,1981-11-29,6c5b7c8add471975aa0fec023b2e8408,1388534355,46.658340,-119.715054,0
555717,2020-12-31 23:59:24,4079773899158,fraud_Breitenberg LLC,travel,7.99,Eric,Preston,M,7020 Doyle Stream Apt. 951,Mesa,...,44.6255,-116.4493,129,Cartographer,1965-12-15,14392d723bb7737606b2700ac791b7aa,1388534364,44.470525,-117.080888,0


In [6]:
fraud_df = test_data.loc[test_data['is_fraud']==1]
fraud_df

,trans_date_trans_time,cc_num,merchant,category,amt,first,last,gender,street,city,...,lat,long,city_pop,job,dob,trans_num,unix_time,merch_lat,merch_long,is_fraud
1685,2020-06-21 22:06:39,3560725013359375,fraud_Hamill-D'Amore,health_fitness,24.84,Brooke,Smith,F,63542 Luna Brook Apt. 012,Notrees,...,31.8599,-102.7413,23,Cytogeneticist,1969-09-15,16bf2e46c54369a8eab2214649506425,1371852399,32.575873,-102.604290,1
1767,2020-06-21 22:32:22,6564459919350820,"fraud_Rodriguez, Yost and Jenkins",misc_net,780.52,Douglas,Willis,M,619 Jeremy Garden Apt. 681,Benton,...,42.5545,-90.3508,1306,Public relations officer,1958-09-10,ab4b379d2c0c9c667d46508d4e126d72,1371853942,42.461127,-91.147148,1
1781,2020-06-21 22:37:27,6564459919350820,fraud_Nienow PLC,entertainment,620.33,Douglas,Willis,M,619 Jeremy Garden Apt. 681,Benton,...,42.5545,-90.3508,1306,Public relations officer,1958-09-10,47a9987ae81d99f7832a54b29a77bf4b,1371854247,42.771834,-90.158365,1
1784,2020-06-21 22:38:55,4005676619255478,"fraud_Heathcote, Yost and Kertzmann",shopping_net,1077.69,William,Perry,M,458 Phillips Island Apt. 768,Denham Springs,...,30.4590,-90.9027,71335,Herbalist,1994-05-31,fe956c7e4a253c437c18918bf96f7b62,1371854335,31.204974,-90.261595,1
1857,2020-06-21 23:02:16,3560725013359375,fraud_Hermann and Sons,shopping_pos,842.65,Brooke,Smith,F,63542 Luna Brook Apt. 012,Notrees,...,31.8599,-102.7413,23,Cytogeneticist,1969-09-15,f6838c01f5d2262006e6b71d33ba7c6d,1371855736,31.315782,-102.736390,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
517197,2020-12-22 22:05:48,2242176657877538,"fraud_Willms, Kris and Bergnaum",shopping_pos,1041.51,Travis,Daniel,M,1327 Rose Causeway Apt. 610,Senatobia,...,34.6323,-89.8855,14462,Database administrator,1959-03-03,35b0297dd026d2e9a75d024a5dec7955,1387749948,34.573471,-89.911011,1
517274,2020-12-22 22:18:07,2242176657877538,fraud_Kuhn LLC,shopping_pos,868.09,Travis,Daniel,M,1327 Rose Causeway Apt. 610,Senatobia,...,34.6323,-89.8855,14462,Database administrator,1959-03-03,da7f67d7375f10a054a3d919448c45dd,1387750687,34.091227,-90.390612,1
517341,2020-12-22 22:31:48,2242176657877538,"fraud_Mosciski, Ziemann and Farrell",shopping_net,1039.42,Travis,Daniel,M,1327 Rose Causeway Apt. 610,Senatobia,...,34.6323,-89.8855,14462,Database administrator,1959-03-03,25b076c7bcd70f272c1c5326bb234f4b,1387751508,34.628434,-90.284780,1
517529,2020-12-22 23:06:03,2242176657877538,fraud_Bauch-Raynor,grocery_pos,289.27,Travis,Daniel,M,1327 Rose Causeway Apt. 610,Senatobia,...,34.6323,-89.8855,14462,Database administrator,1959-03-03,2df7d894868fbc99ec1d8b055585fc9d,1387753563,34.746063,-90.401093,1


In [7]:
temp_df_1 = genuine_df.loc[:5000]
temp_df_2 =  fraud_df.head(500)
test_dataset_1 = temp_df_1
test_dataset_1 = test_dataset_1.append(temp_df_2)
print(len(test_dataset_1))
test_dataset_1.head(3)

5483


,trans_date_trans_time,cc_num,merchant,category,amt,first,last,gender,street,city,...,lat,long,city_pop,job,dob,trans_num,unix_time,merch_lat,merch_long,is_fraud
0,2020-06-21 12:14:25,2291163933867244,fraud_Kirlin and Sons,personal_care,2.86,Jeff,Elliott,M,351 Darlene Green,Columbia,...,33.9659,-80.9355,333497,Mechanical engineer,1968-03-19,2da90c7d74bd46a0caf3777415b3ebd3,1371816865,33.986391,-81.200714,0
1,2020-06-21 12:14:33,3573030041201292,fraud_Sporer-Keebler,personal_care,29.84,Joanne,Williams,F,3638 Marsh Union,Altonah,...,40.3207,-110.4360,302,"Sales professional, IT",1990-01-17,324cc204407e99f51b0d6ca0055005e7,1371816873,39.450498,-109.960431,0
2,2020-06-21 12:14:53,3598215285024754,"fraud_Swaniawski, Nitzsche and Welch",health_fitness,41.28,Ashley,Lopez,F,9333 Valentine Point,Bellmore,...,40.6729,-73.5365,34496,"Librarian, public",1970-10-21,c81755dbbbea9d5c77f094348a7579be,1371816893,40.495810,-74.196111,0


In [8]:
temp_df_1 = genuine_df.loc[6000:7500]
temp_df_2 =  fraud_df.tail(1000)
test_dataset_2 = temp_df_1
test_dataset_2 = test_dataset_2.append(temp_df_2)
print(len(test_dataset_2))
test_dataset_2.head()

2500


,trans_date_trans_time,cc_num,merchant,category,amt,first,last,gender,street,city,...,lat,long,city_pop,job,dob,trans_num,unix_time,merch_lat,merch_long,is_fraud
6000,2020-06-22 23:28:37,4756039869079882102,fraud_Berge-Hills,kids_pets,99.41,Francisco,Hernandez,M,980 Smith Gardens,Gainesville,...,33.6547,-97.1583,26120,"Engineer, manufacturing",1954-01-06,4b6590ab98d9856386b2b75453e1d7a9,1371943717,32.855236,-97.404431,0
6001,2020-06-22 23:29:04,4683520018489354,"fraud_Baumbach, Strosin and Nicolas",shopping_pos,9.28,Charles,Carrillo,M,17547 Stephen Turnpike Apt. 846,Hawthorne,...,33.9143,-118.3493,93193,"Editor, magazine features",1995-04-19,736feae2face7599454fb797cacfa834,1371943744,34.227162,-117.364824,0
6002,2020-06-22 23:29:49,4841313716651064,fraud_Abshire PLC,entertainment,77.36,Omar,Gay,M,2161 Blankenship Freeway,West Hartford,...,43.7185,-72.4439,140,"Development worker, community",1989-10-28,a85a00adf4cdde0024c83ea0bd68bbb7,1371943789,43.945313,-72.153559,0
6003,2020-06-22 23:29:49,30235438713303,fraud_Gerlach Inc,shopping_net,56.52,James,Baldwin,M,3603 Mitchell Court,Winfield,...,38.5072,-81.8900,5512,Exhibition designer,1980-03-24,a90fe21e7d0b0ca62bdd8bd6f6fba37f,1371943789,37.588632,-82.221931,0
6004,2020-06-22 23:30:02,501802953619,"fraud_Runte, Green and Emard",health_fitness,46.13,Robert,Flores,M,3277 Fields Meadows Apt. 790,Greenview,...,41.5403,-122.9366,308,Call centre manager,1958-09-20,70731640d4eb7ed9e861e7599597902d,1371943802,40.963244,-122.614452,0


In [9]:
test_dataset_1.to_csv("../exporteddata/creditcard_test1.csv")
test_dataset_2.to_csv("../exporteddata/creditcard_test2.csv")

In [10]:
sqlstmt = 'CREATE TABLE public.creditcard_test1 (index bigint NOT NULL GENERATED ALWAYS AS IDENTITY, \
    trans_date_trans_time text COLLATE pg_catalog."default", \
    cc_num bigint, merchant text COLLATE pg_catalog."default",\
    category text COLLATE pg_catalog."default",\
    amt double precision, first text COLLATE pg_catalog."default",\
    last text COLLATE pg_catalog."default",\
    gender text COLLATE pg_catalog."default",\
    street text COLLATE pg_catalog."default",\
    city text COLLATE pg_catalog."default",\
    state text COLLATE pg_catalog."default",\
    zip bigint, lat double precision, "long" double precision, city_pop bigint,\
    job text COLLATE pg_catalog."default", dob text COLLATE pg_catalog."default", \
    trans_num text COLLATE pg_catalog."default", unix_time bigint, \
    merch_lat double precision, merch_long double precision, is_fraud bigint )'

In [11]:
import sys
sys.path.insert(0, '../')
from config import username, passowrd

connection_string = f'{username}:{passowrd}@localhost:5432/Creditcard_db'   
engine = create_engine(f'postgresql://{connection_string}')

engine.execute("DROP TABLE IF EXISTS creditcard_test1")

engine.execute(sqlstmt)

In [12]:
# engine.execute("drop table if exists creditcard_test2")

test_dataset_1.to_sql('creditcard_test1', con=engine, if_exists='append', index=False)

In [13]:
engine.execute("ALTER TABLE public.creditcard_test1 ADD CONSTRAINT PK_index PRIMARY KEY (index)")

In [14]:
# test_dataset_2.to_sql('creditcard_test2', con=engine, if_exists='append', index=False)

## Clean the data

In [15]:
# Convert the date of birth into numeric as age
data_train = test_dataset_1
data_train['dob']= pd.to_datetime(data_train['dob'])
data_train['dob']

from datetime import datetime
today = datetime.today()
data_train['age'] = round(data_train['dob'].apply(lambda x: (today - x).days//365.25),0)
data_train['age']

0         52.0
1         30.0
2         50.0
3         33.0
4         65.0
          ... 
125489    96.0
125496    21.0
125527    26.0
125530    26.0
125550    21.0
Name: age, Length: 5483, dtype: float64

In [16]:
# Select the columns which are required for the further processing
data_train = data_train[["category", "cc_num", "amt", "lat","long", "job", "age", "trans_num", 
                         "unix_time", "merch_lat","merch_long", "is_fraud"]]
data_train

,category,cc_num,amt,lat,long,job,age,trans_num,unix_time,merch_lat,merch_long,is_fraud
0,personal_care,2291163933867244,2.86,33.9659,-80.9355,Mechanical engineer,52.0,2da90c7d74bd46a0caf3777415b3ebd3,1371816865,33.986391,-81.200714,0
1,personal_care,3573030041201292,29.84,40.3207,-110.4360,"Sales professional, IT",30.0,324cc204407e99f51b0d6ca0055005e7,1371816873,39.450498,-109.960431,0
2,health_fitness,3598215285024754,41.28,40.6729,-73.5365,"Librarian, public",50.0,c81755dbbbea9d5c77f094348a7579be,1371816893,40.495810,-74.196111,0
3,misc_pos,3591919803438423,60.05,28.5697,-80.8191,Set designer,33.0,2159175b9efe66dc301f149d3d5abf8c,1371816915,28.812398,-80.883061,0
4,travel,3526826139003047,3.19,44.2529,-85.0170,Furniture designer,65.0,57ff021bd3f328f8738bb535c302a31b,1371816917,44.959148,-85.884734,0
...,...,...,...,...,...,...,...,...,...,...,...,...
125489,shopping_net,2358122155477959,1037.21,39.2092,-76.6681,"Conservator, furniture",96.0,10ad75eeb90c45a7f64543463a52d998,1375570081,39.167276,-77.027134,1
125496,misc_pos,4653879239169997,8.77,37.0581,-86.1938,"Therapist, sports",21.0,a615441e7642b7d1c9983da976cbb919,1375570241,37.810239,-85.765720,1
125527,home,4725837176265195730,292.85,30.8985,-87.1332,Higher education careers adviser,26.0,8e6f55ac39c62d05e869df64294f128c,1375570860,29.952503,-86.844989,1
125530,health_fitness,4725837176265195730,19.79,30.8985,-87.1332,Higher education careers adviser,26.0,6f4e4cb209429e6e26325debaf05924a,1375570933,31.692756,-86.176423,1


### One Hot Encoding

In [17]:
# Encode the transaction number and convert into numeric
from sklearn.preprocessing import LabelEncoder

get_transnum = data_train['trans_num']
label_encoder_t = LabelEncoder()
label_encoder_t.fit(get_transnum)
encoded_transnum = label_encoder_t.transform(get_transnum)
data_train['trans_num'] = encoded_transnum

C:\ProgramData\Anaconda3\envs\P36\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [18]:
# Encode the category and convert into numeric
get_category = data_train['category']
label_encoder_c = LabelEncoder()
label_encoder_c.fit(get_category)
encoded_category = label_encoder_c.transform(get_category)
data_train['category'] = encoded_category

C:\ProgramData\Anaconda3\envs\P36\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [19]:
get_category

0          personal_care
1          personal_care
2         health_fitness
3               misc_pos
4                 travel
               ...      
125489      shopping_net
125496          misc_pos
125527              home
125530    health_fitness
125550          misc_pos
Name: category, Length: 5483, dtype: object

In [20]:
# Encode the job and convert into numeric
get_job = data_train['job']
label_encoder_j = LabelEncoder()
label_encoder_j.fit(get_job)
encoded_job = label_encoder_j.transform(get_job)
data_train['job'] = encoded_job

C:\ProgramData\Anaconda3\envs\P36\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [21]:
data_train

,category,cc_num,amt,lat,long,job,age,trans_num,unix_time,merch_lat,merch_long,is_fraud
0,10,2291163933867244,2.86,33.9659,-80.9355,272,52.0,982,1371816865,33.986391,-81.200714,0
1,10,3573030041201292,29.84,40.3207,-110.4360,386,30.0,1081,1371816873,39.450498,-109.960431,0
2,5,3598215285024754,41.28,40.6729,-73.5365,256,50.0,4331,1371816893,40.495810,-74.196111,0
3,9,3591919803438423,60.05,28.5697,-80.8191,401,33.0,714,1371816915,28.812398,-80.883061,0
4,13,3526826139003047,3.19,44.2529,-85.0170,193,65.0,1874,1371816917,44.959148,-85.884734,0
...,...,...,...,...,...,...,...,...,...,...,...,...
125489,11,2358122155477959,1037.21,39.2092,-76.6681,96,96.0,359,1375570081,39.167276,-77.027134,1
125496,9,4653879239169997,8.77,37.0581,-86.1938,447,21.0,3578,1375570241,37.810239,-85.765720,1
125527,6,4725837176265195730,292.85,30.8985,-87.1332,214,26.0,3052,1375570860,29.952503,-86.844989,1
125530,5,4725837176265195730,19.79,30.8985,-87.1332,214,26.0,2359,1375570933,31.692756,-86.176423,1


### Connect to postgres database

In [22]:
from config import username, passowrd

connection_string = f'{username}:{passowrd}@localhost:5432/Creditcard_db'   
engine = create_engine(f'postgresql://{connection_string}')

In [45]:
data_train

,category,cc_num,amt,lat,long,job,age,trans_num,unix_time,merch_lat,merch_long,is_fraud
0,10,2291163933867244,2.86,33.9659,-80.9355,272,52.0,982,1371816865,33.986391,-81.200714,0
1,10,3573030041201292,29.84,40.3207,-110.4360,386,30.0,1081,1371816873,39.450498,-109.960431,0
2,5,3598215285024754,41.28,40.6729,-73.5365,256,50.0,4331,1371816893,40.495810,-74.196111,0
3,9,3591919803438423,60.05,28.5697,-80.8191,401,33.0,714,1371816915,28.812398,-80.883061,0
4,13,3526826139003047,3.19,44.2529,-85.0170,193,65.0,1874,1371816917,44.959148,-85.884734,0
...,...,...,...,...,...,...,...,...,...,...,...,...
125489,11,2358122155477959,1037.21,39.2092,-76.6681,96,96.0,359,1375570081,39.167276,-77.027134,1
125496,9,4653879239169997,8.77,37.0581,-86.1938,447,21.0,3578,1375570241,37.810239,-85.765720,1
125527,6,4725837176265195730,292.85,30.8985,-87.1332,214,26.0,3052,1375570860,29.952503,-86.844989,1
125530,5,4725837176265195730,19.79,30.8985,-87.1332,214,26.0,2359,1375570933,31.692756,-86.176423,1


In [46]:
engine.execute("drop table if exists creditcard_test")

data_train.to_sql('creditcard_test', con=engine, if_exists='append', index=False)

In [47]:
engine.table_names()

['creditcard_test1', 'creditcard_test']

## Test the data with the saved model

In [48]:
# Select all the records from the table
sql_stmt = "select * from creditcard_test"

result_set = engine.execute(sql_stmt)

results = []

for r in result_set:  
    results.append(r)
    print(r)    

(10, 2291163933867244, 2.86, 33.9659, -80.9355, 272, 52.0, 982, 1371816865, 33.986391, -81.200714, 0)
(10, 3573030041201292, 29.84, 40.3207, -110.436, 386, 30.0, 1081, 1371816873, 39.450498, -109.960431, 0)
(5, 3598215285024754, 41.28, 40.6729, -73.5365, 256, 50.0, 4331, 1371816893, 40.49581, -74.196111, 0)
(9, 3591919803438423, 60.05, 28.5697, -80.8191, 401, 33.0, 714, 1371816915, 28.812398, -80.883061, 0)
(13, 3526826139003047, 3.19, 44.2529, -85.01700000000002, 193, 65.0, 1874, 1371816917, 44.959148, -85.884734, 0)
(7, 30407675418785, 19.55, 42.1939, -76.7361, 355, 29.0, 2608, 1371816937, 41.747157, -77.584197, 0)
(5, 213180742685905, 133.93, 40.507, -123.9743, 446, 69.0, 498, 1371816944, 41.499458, -124.888729, 0)
(10, 3589289942931264, 10.37, 43.7557, -97.5936, 121, 48.0, 2997, 1371816950, 44.495498, -97.728453, 0)
(12, 3596357274378601, 4.37, 41.0001, -78.2357, 12, 47.0, 2423, 1371816970, 41.546067, -78.120238, 0)
(1, 3546897637165774, 66.54, 31.6591, -96.8094, 39, 64.0, 3618, 13

(3, 30266994494236, 70.93, 37.1349, -77.7364, 69, 43.0, 5294, 1371864405, 37.267809, -77.588892, 0)
(2, 4469777115158234880, 63.51, 42.728, -71.181, 269, 39.0, 3212, 1371864453, 43.491347, -70.875603, 0)
(11, 60427851591, 98.38, 35.3492, -99.188, 328, 44.0, 4965, 1371864522, 36.142628, -99.155859, 0)
(8, 502012776709, 9.08, 42.6315, -75.1866, 292, 75.0, 99, 1371864550, 43.47100800000001, -75.585414, 0)
(4, 375974680629816, 81.19, 41.6611, -73.0449, 213, 47.0, 3134, 1371864555, 41.716265, -72.672775, 0)
(2, 3596217206093829, 62.6, 40.8626, -91.9534, 54, 32.0, 1087, 1371864556, 41.035919, -91.257712, 0)
(12, 5456776410929288, 5.86, 41.0472, -79.8089, 311, 66.0, 2748, 1371864578, 41.472327, -80.76031, 0)
(2, 3506042666828517, 62.83, 38.1372, -81.5962, 186, 61.0, 956, 1371864593, 38.114718, -81.542048, 0)
(4, 3558652751678952, 72.19, 32.6205, -94.2, 74, 89.0, 3701, 1371864595, 31.906456, -94.371546, 0)
(0, 2233882705243596, 74.89, 33.357, -89.0473, 275, 60.0, 5288, 1371864604, 33.436753, -

(1, 38797410705641, 48.81, 42.4069, -74.1528, 205, 48.0, 1868, 1371908711, 42.54146, -73.512996, 0)
(0, 30270432095985, 99.61, 40.6761, -91.0391, 168, 47.0, 4832, 1371908742, 41.133415, -90.831145, 0)
(7, 676369110710, 24.18, 42.4969, -83.2911, 145, 45.0, 3878, 1371908753, 42.96530900000001, -83.80669, 0)
(7, 3514865930894695, 11.86, 32.9396, -105.8189, 292, 53.0, 1454, 1371908758, 32.023898, -105.138853, 0)
(1, 6515067575568369, 31.82, 36.6403, -81.7776, 246, 30.0, 4564, 1371908795, 37.579356, -82.334476, 0)
(6, 4155021259183870, 5.74, 34.7032, -98.9591, 374, 37.0, 4635, 1371908806, 34.076658, -99.096024, 0)
(7, 3554849923339851, 117.55, 36.1043, -97.0609, 52, 72.0, 778, 1371908836, 36.725289, -97.806565, 0)
(11, 4198470814557, 5.2, 41.4768, -95.3509, 165, 49.0, 806, 1371908861, 41.846987, -95.003379, 0)
(5, 2242542703101233, 17.62, 38.4921, -85.4524, 314, 24.0, 5358, 1371908861, 38.213306, -86.091533, 0)
(5, 213195754007681, 5.78, 41.2001, -92.1354, 85, 51.0, 1421, 1371908886, 41.272

In [49]:
result_set_df = pd.DataFrame(results, columns=["category", "cc_num", "amt", "lat","long", "job", "age", "trans_num", 
                         "unix_time", "merch_lat","merch_long", "is_fraud"])
result_set_df

,category,cc_num,amt,lat,long,job,age,trans_num,unix_time,merch_lat,merch_long,is_fraud
0,10,2291163933867244,2.86,33.9659,-80.9355,272,52.0,982,1371816865,33.986391,-81.200714,0
1,10,3573030041201292,29.84,40.3207,-110.4360,386,30.0,1081,1371816873,39.450498,-109.960431,0
2,5,3598215285024754,41.28,40.6729,-73.5365,256,50.0,4331,1371816893,40.495810,-74.196111,0
3,9,3591919803438423,60.05,28.5697,-80.8191,401,33.0,714,1371816915,28.812398,-80.883061,0
4,13,3526826139003047,3.19,44.2529,-85.0170,193,65.0,1874,1371816917,44.959148,-85.884734,0
...,...,...,...,...,...,...,...,...,...,...,...,...
5478,11,2358122155477959,1037.21,39.2092,-76.6681,96,96.0,359,1375570081,39.167276,-77.027134,1
5479,9,4653879239169997,8.77,37.0581,-86.1938,447,21.0,3578,1375570241,37.810239,-85.765720,1
5480,6,4725837176265195730,292.85,30.8985,-87.1332,214,26.0,3052,1375570860,29.952503,-86.844989,1
5481,5,4725837176265195730,19.79,30.8985,-87.1332,214,26.0,2359,1375570933,31.692756,-86.176423,1


In [50]:
label_encoder_c.classes_

array(['entertainment', 'food_dining', 'gas_transport', 'grocery_net',
       'grocery_pos', 'health_fitness', 'home', 'kids_pets', 'misc_net',
       'misc_pos', 'personal_care', 'shopping_net', 'shopping_pos',
       'travel'], dtype=object)

In [51]:
# Define the features
selected_features = result_set_df.drop('is_fraud', axis=1)

# sets y to is_fraud
target = result_set_df["is_fraud"].values.reshape(-1, 1)

In [52]:
# Scale the train & test datasets
from sklearn.preprocessing import StandardScaler

# Create a standard scaler model and fit it to the training data
X_scaler = StandardScaler().fit(selected_features)

# Transform the scaled data
X_test_scaled = X_scaler.transform(selected_features)

In [53]:
import joblib
# filename = "logistic_regn_model.sav"
# filename = "gnb_model.sav"

filename = "logistic_balanced_model.sav"

# load the model from disk
loaded_model = joblib.load(filename)

predictions = loaded_model.predict(X_test_scaled)

In [54]:
print(f"First 10 Predictions:   {predictions[:10]}")
# print(f"First 10 Actual labels: {target[:10].tolist()}")
print(f"First 10 Actual labels: {target[:10]}")

First 10 Predictions:   [0 0 0 0 0 0 1 0 0 0]
First 10 Actual labels: [[0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]]


## Display the output into a dataframe

In [55]:
y_predictions = loaded_model.predict(X_test_scaled)

actual_value = []
predicted_value = []

for i in range(len(target)):
    target_item = target[i][0]
#     print(f"{target_item}    {target[i]}")
    actual_value.append(target_item)
    
for i in range(0, len(y_predictions)):
    pred_item = y_predictions[i]
    predicted_value.append(pred_item)

In [56]:
label_encoder_c.inverse_transform(encoded_category)

array(['personal_care', 'personal_care', 'health_fitness', ..., 'home',
       'health_fitness', 'misc_pos'], dtype=object)

In [57]:
output_csv = pd.DataFrame()

output_csv["category"] = label_encoder_c.inverse_transform(encoded_category)
output_csv["job"] = label_encoder_j.inverse_transform(encoded_job)
output_csv["trans_num"] = label_encoder_t.inverse_transform(encoded_transnum)
output_csv["amount"] = selected_features['amt']
output_csv["unixtime"] = selected_features['unix_time']
output_csv["actual value"] = actual_value
output_csv["predicted value"] = predicted_value

output_csv


,category,job,trans_num,amount,unixtime,actual value,predicted value
0,personal_care,Mechanical engineer,2da90c7d74bd46a0caf3777415b3ebd3,2.86,1371816865,0,0
1,personal_care,"Sales professional, IT",324cc204407e99f51b0d6ca0055005e7,29.84,1371816873,0,0
2,health_fitness,"Librarian, public",c81755dbbbea9d5c77f094348a7579be,41.28,1371816893,0,0
3,misc_pos,Set designer,2159175b9efe66dc301f149d3d5abf8c,60.05,1371816915,0,0
4,travel,Furniture designer,57ff021bd3f328f8738bb535c302a31b,3.19,1371816917,0,0
...,...,...,...,...,...,...,...
5478,shopping_net,"Conservator, furniture",10ad75eeb90c45a7f64543463a52d998,1037.21,1375570081,1,1
5479,misc_pos,"Therapist, sports",a615441e7642b7d1c9983da976cbb919,8.77,1375570241,1,0
5480,home,Higher education careers adviser,8e6f55ac39c62d05e869df64294f128c,292.85,1375570860,1,1
5481,health_fitness,Higher education careers adviser,6f4e4cb209429e6e26325debaf05924a,19.79,1375570933,1,0


In [58]:
output_csv[-20:]

,category,job,trans_num,amount,unixtime,actual value,predicted value
5463,entertainment,Higher education careers adviser,058f95bbb994f9d2a1e9dd7b3c09685f,532.58,1375493725,1,1
5464,misc_pos,Visual merchandiser,f62547fed07b5d444ca8e53cdbeae2eb,684.31,1375494233,1,1
5465,grocery_pos,"Conservator, furniture",1d629d8f67a23b4a77c14762264e842a,337.71,1375494487,1,1
5466,misc_net,Visual merchandiser,821bf185d401a4936ddf637e5ac35bc8,772.78,1375494622,1,1
5467,grocery_pos,"Therapist, sports",489810f0c35c7fada46261d3e48f18ee,319.08,1375496380,1,1
5468,shopping_net,Higher education careers adviser,07164e1456e4bbfa6808060f279fa5e8,987.01,1375497027,1,1
5469,shopping_net,Higher education careers adviser,0e7d5f5916e4ef8a3be654201eedb94f,906.20,1375497254,1,1
5470,grocery_pos,"Therapist, sports",a22b1e3492a4aec8cc15073d82779f34,301.61,1375498877,1,1
5471,grocery_pos,Visual merchandiser,09b2730549c56b66240b040dfd2773f8,332.09,1375499933,1,1
5472,grocery_net,"Conservator, furniture",05c58f4cc15ecc450c646f6ebb9d8f5b,9.69,1375501758,1,0


In [59]:
output_csv.to_html("../templates/output_dframe.html")